<a href="https://colab.research.google.com/github/JosueBorchardt/Convolucao_2D/blob/main/Convolu%C3%A7%C3%A3o_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolução 2D em Tempo Discreto

## Bibliotecas

In [20]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image
import requests
from io import BytesIO

## Funções para carregar e exibir a imagem

In [21]:
def load_image_from_url(url):
  response = requests.get(url)

  #Converte para escala de cinza
  image = Image.open(BytesIO(response.content)).convert('L')

  #Converte para array NumPy
  image_array = np.array(image)

  # Normaliza os valores para o intervalo [0, 255]
  image_array = ((image_array - image_array.min()) / (image_array.max() - image_array.min())) * 255

  return image_array

In [22]:
def plot_image(image_array, title):
  image_array = np.flipud(image_array)
  fig = go.Figure()
  fig.add_trace(go.Heatmap(z = image_array, colorscale = 'gray'))
  fig.update_layout(
      title=title,
      xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
      yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
  )
  fig.show()

## Convolução 2D

In [23]:
def convolution_2d(image, kernel):
  # Obter dimensões da imagem e do kernel
  img_height, img_width = image.shape
  kernel_height, kernel_width = kernel.shape

  # Calcular os deslocamentos para o kernel
  pad_height = kernel_height // 2
  pad_width = kernel_width // 2

  # Inicializar a imagem de saída
  output_image = np.zeros_like(image)

  # Aplicar convolução
  for i in range(pad_height, img_height - pad_height):
      for j in range(pad_width, img_width - pad_width):
          region = image[i - pad_height:i + pad_height + 1, j - pad_width:j + pad_width + 1]
          output_image[i, j] = np.sum(region * kernel)

  return output_image

## Kernel

In [24]:
#Kernel de suavização
K_smoothing = np.array([[1, 1, 1],
                        [1, 1, 1],
                        [1, 1, 1]])/9.0

#Kernel de bordas
K_edges = np.array([[-1, -1, -1],
                    [-1,  8, -1],
                    [-1, -1, -1]])

## Filtragem

In [ ]:
#A imagem utilizada no projeto foi retirada do Unsplash
image = load_image_from_url('https://images.unsplash.com/photo-1727163941293-0c3dc5074bf4?q=80&w=1632&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D')
smoothed_image = convolution_2d(image, K_smoothing)
edges_image = convolution_2d(smoothed_image, K_edges)
edges_image = np.clip(edges_image, 0, 255)

plot_image(image, "Imagem original")
plot_image(smoothed_image, "Imagem suavizada")
plot_image(edges_image, "Imagem com detecção de bordas")